In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import os
os.getcwd()

data =  pd.read_csv('train.csv')

In [6]:
#find NaN values per column

data.isna().sum().sort_values

<bound method Series.sort_values of Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual        

In [7]:
#get dummies in categorical features

data2 = pd.get_dummies(data)

##drop columns that have many NaN 
data2 = data2.drop('LotFrontage', axis=1)
data2 = data2.drop('GarageYrBlt', axis=1)

##fill MansVnrArea where NaN values with 0
data2.MasVnrArea = data2.MasVnrArea.fillna(0)


In [8]:
#check if there are any NaN values after the imputing 

data2.isna().sum().sort_values

<bound method Series.sort_values of Id                       0
MSSubClass               0
LotArea                  0
OverallQual              0
OverallCond              0
YearBuilt                0
YearRemodAdd             0
MasVnrArea               0
BsmtFinSF1               0
BsmtFinSF2               0
BsmtUnfSF                0
TotalBsmtSF              0
1stFlrSF                 0
2ndFlrSF                 0
LowQualFinSF             0
GrLivArea                0
BsmtFullBath             0
BsmtHalfBath             0
FullBath                 0
HalfBath                 0
BedroomAbvGr             0
KitchenAbvGr             0
TotRmsAbvGrd             0
Fireplaces               0
GarageCars               0
GarageArea               0
WoodDeckSF               0
OpenPorchSF              0
EnclosedPorch            0
3SsnPorch                0
                        ..
GarageCond_TA            0
PavedDrive_N             0
PavedDrive_P             0
PavedDrive_Y             0
PoolQC_Ex          

In [9]:
#split train and test sets for X and y

from sklearn.model_selection import train_test_split

y=data2['SalePrice']

X=data2.drop('SalePrice', axis=1)

X_train,X_test ,y_train,y_test= train_test_split(X,y,test_size=0.25,random_state=5)

In [10]:
#shape of X and y and train and test sets
print(X_train.shape,y_train.shape)

print(X_test.shape,y_test.shape)

((1095, 287), (1095L,))
((365, 287), (365L,))


In [11]:
#train model with Linear Regression

from sklearn.linear_model import LinearRegression

lr = LinearRegression()  

lr.fit(X_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [12]:
#predict SalesPrices with LinearRegression

preds = lr.predict(X_test)

In [13]:
import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y_pred, y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))


In [14]:
# MlSE using LinearRegression

error = rmsle(preds, y_test)

error

0.14973853543232002

In [15]:
# feature selection using Lasso
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lass = Lasso(alpha=0.5)  # alpha determines the strength of the regularization
                          # the higer the value, the more coefficients will be zero
lass.fit(X_train, y_train)

from sklearn.feature_selection import SelectFromModel
lasso_selector = SelectFromModel(lass, prefit=True)  # prefit because we have already trained the model
X_selected = lasso_selector.transform(X_train) 
X_testl = lasso_selector.transform(X_test)
# plain transform because model is already fit
print('shape of Xtest after selection:', X_testl.shape)
print('shape after selection:', X_selected.shape)


('shape of Xtest after selection:', (365L, 261L))
('shape after selection:', (1095L, 261L))


C:\Users\maria\Anaconda2\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [16]:
# prediction with LinearRegression after feature selection using Lasso

lr.fit(X_selected, y_train)
preds1 = lr.predict(X_testl)


In [17]:
# error after Lasso
error = rmsle(preds1, y_test)

error

0.14927553329461699

In [18]:
## predict SalePrice usin LR, SVM, Decision Tree, Neural Net, KNN, Random Forest without Lasso

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from math import sqrt

#LR
reg =linear_model.LinearRegression()
reg.fit(X_train, y_train)
pred_LR = reg.predict(X_test)
print("LR:", rmsle(pred_LR,y_test))

# SVM
svc=SVR(C=1.0, epsilon=0.2)
svc.fit(X_train, y_train)
pred_SVM = svc.predict(X_test)
print("SVR:", rmsle(pred_SVM,y_test))

# Decision Tree
decision_tree = DecisionTreeRegressor(random_state=21)
decision_tree.fit(X_train, y_train)
predictions_DT = decision_tree.predict(X_test)
print("Decision Tree:", rmsle(predictions_DT,y_test))

# # Neural Net
nn = MLPRegressor(hidden_layer_sizes=(10,),  activation='relu', solver='adam',    alpha=0.001,batch_size='auto',
              learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
              random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
              nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
              epsilon=1e-08)
nn.fit(X_train, y_train)
pred_nn = nn.predict(X_test)
print("NN:", rmsle(pred_nn,y_test))

# KNN
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train,y_train)
predictions_KNN = knn.predict(X_test)
print("KNN:", rmsle(predictions_KNN,y_test))

# Random Forest
rf = RandomForestRegressor(max_depth=2, random_state=0)
rf.fit(X_train,y_train)
predictions_RF = rf.predict(X_test)
print("Random Forest:", rmsle(predictions_RF,y_test))

('LR:', 0.14973853543232002)
('SVR:', 0.4076445132324007)
('Decision Tree:', 0.21727675307494657)
('NN:', 0.2157577700229097)
('KNN:', 0.2324149598376215)
('Random Forest:', 0.2725554687316167)


In [19]:
## predict SalePrice usin LR, SVM, Decision Tree, Neural Net, KNN, Random Forest after Lasso

#LR
reg =linear_model.LinearRegression()
reg.fit(X_selected, y_train)
pred_LR = reg.predict(X_testl)
print("LR:", rmsle(pred_LR,y_test))

# SVM
svc=SVR(C=1.0, epsilon=0.2)
svc.fit(X_selected, y_train)
pred_SVM = svc.predict(X_testl)
print("SVR:", rmsle(pred_SVM,y_test))

# Decision Tree
decision_tree = DecisionTreeRegressor(random_state=21)
decision_tree.fit(X_selected, y_train)
predictions_DT = decision_tree.predict(X_testl)
print("Decision Tree:", rmsle(predictions_DT,y_test))

# # Neural Net
nn = MLPRegressor(hidden_layer_sizes=(10,),  activation='relu', solver='adam',    alpha=0.001,batch_size='auto',
              learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
              random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
              nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
              epsilon=1e-08)
nn.fit(X_selected, y_train)
pred_nn = nn.predict(X_testl)
print("NN:", rmsle(pred_nn,y_test))

# KNN
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_selected,y_train)
predictions_KNN = knn.predict(X_testl)
print("KNN:", rmsle(predictions_KNN,y_test))

# Random Forest
rf = RandomForestRegressor(max_depth=2, random_state=0)
rf.fit(X_selected,y_train)
predictions_RF = rf.predict(X_testl)
print("Random Forest:", rmsle(predictions_RF,y_test))

('LR:', 0.14927553329461699)
('SVR:', 0.4076445132729237)
('Decision Tree:', 0.2031993951042532)
('NN:', 0.25125246092718606)
('KNN:', 0.2324149598376215)
('Random Forest:', 0.2725554687316167)


In [20]:
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

lass = linear_model.Lasso()

grid = [{'alpha':np.arange(0.01, 1.0, 0.01)}]

reg = GridSearchCV(lass, grid, cv=5)

reg.fit(X_train,y_train)
preds_lasso = reg.predict(X_test)

error = rmsle(preds_lasso,y_test)

error

# # reg.best_estimator_

0.13223814796772418

In [21]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

def model_extra_trees_regression(Xtrain,Xtest,ytrain,y_test):
    
    X_train = Xtrain
    y_train = ytrain
    
    etr = ExtraTreesRegressor(n_jobs=1, random_state=77)
    param_grid = {'max_features': [0.01,1,5]}
    model = GridSearchCV(estimator=etr, param_grid=param_grid, n_jobs=1, cv=10)
    model.fit(X_train, y_train)
    print('Extra trees regression...')
    print('Best Params:')
    print(model.best_params_)
    #print('Best CV Score:')
    #print(-model.best_score_)

    y_pred = model.predict(Xtest)
    return  rmsle(y_pred,y_test)

In [22]:
model_extra_trees_regression(X_train,X_test,y_train,y_test)

Extra trees regression...
Best Params:
{'max_features': 5}


0.20664001071824753